[View in Colaboratory](https://colab.research.google.com/github/mayojich/TensorFlow/blob/master/Building%20Input%20Functions%20with%20tf.contrib.learn.ipynb)

[Building Input Functions with tf.contrib.learn](https://www.tensorflow.org/versions/r1.2/get_started/input_fn)

**CUSTOM INPUT PIPELINE WITH INPUT_FN**

In [0]:
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING, target_dtype=np.int, features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST, target_dtype=np.int, features_dtype=np.float32)
...

classifier.fit(x=training_set.data,
               y=training_set.target,
               steps=2000)

**Anatomy of an input_fn**

In [0]:
def my_input_fn():

    # Preprocess your data here...

    # ...then return 1) a mapping of feature columns to Tensors with
    # the corresponding feature data, and 2) a Tensor containing labels
    return feature_cols, labels

**Converting Feature Data to Tensors**

In [0]:
feature_column_data = [1, 2.4, 0, 9.9, 3, 120]
feature_tensor = tf.constant(feature_column_data)

In [0]:
sparse_tensor = tf.SparseTensor(indices=[[0,1], [2,4]],
                                values=[6, 0.5],
                                dense_shape=[3, 5])

In [0]:
[[0, 6, 0, 0, 0]
 [0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0.5]]

**Passing input_fn Data to Your Model**


In [0]:
classifier.fit(input_fn=my_input_fn, steps=2000)

In [0]:
classifier.fit(input_fn=my_input_fn(training_set), steps=2000)

In [0]:
def my_input_function_training_set():
  return my_input_function(training_set)

classifier.fit(input_fn=my_input_fn_training_set, steps=2000)

In [0]:
classifier.fit(input_fn=functools.partial(my_input_function,
                                          data_set=training_set), steps=2000)

In [0]:
classifier.fit(input_fn=lambda: my_input_fn(training_set), steps=2000)

In [0]:
classifier.evaluate(input_fn=lambda: my_input_fn(test_set), steps=2000)

**A Neural Network Model for Boston House Values**


**SETUP**


**Importing the Housing Data**

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [0]:
COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age",
           "dis", "tax", "ptratio", "medv"]
FEATURES = ["crim", "zn", "indus", "nox", "rm",
            "age", "dis", "tax", "ptratio"]
LABEL = "medv"

training_set = pd.read_csv("boston_train.csv", skipinitialspace=True,
                           skiprows=1, names=COLUMNS)
test_set = pd.read_csv("boston_test.csv", skipinitialspace=True,
                       skiprows=1, names=COLUMNS)
prediction_set = pd.read_csv("boston_predict.csv", skipinitialspace=True,
                             skiprows=1, names=COLUMNS)

**Defining FeatureColumns and Creating the Regressor**

In [0]:
feature_cols = [tf.contrib.layers.real_valued_column(k)
                  for k in FEATURES]

In [0]:
regressor = tf.contrib.learn.DNNRegressor(feature_columns=feature_cols,
                                          hidden_units=[10, 10],
                                          model_dir="/tmp/boston_model")

**Building the input_fn**


In [0]:
def input_fn(data_set):
  feature_cols = {k: tf.constant(data_set[k].values)
                  for k in FEATURES}
  labels = tf.constant(data_set[LABEL].values)
  return feature_cols, labels

**Training the Regressor**


In [0]:
regressor.fit(input_fn=lambda: input_fn(training_set), steps=5000)

In [0]:
INFO:tensorflow:Step 1: loss = 483.179
INFO:tensorflow:Step 101: loss = 81.2072
INFO:tensorflow:Step 201: loss = 72.4354
...
INFO:tensorflow:Step 1801: loss = 33.4454
INFO:tensorflow:Step 1901: loss = 32.3397
INFO:tensorflow:Step 2001: loss = 32.0053
INFO:tensorflow:Step 4801: loss = 27.2791
INFO:tensorflow:Step 4901: loss = 27.2251
INFO:tensorflow:Saving checkpoints for 5000 into /tmp/boston_model/model.ckpt.
INFO:tensorflow:Loss for final step: 27.1674.

**Evaluating the Model**


In [0]:
ev = regressor.evaluate(input_fn=lambda: input_fn(test_set), steps=1)

In [0]:
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))

In [0]:
INFO:tensorflow:Eval steps [0,1) for training step 5000.
INFO:tensorflow:Saving evaluation summary for 5000 step: loss = 11.9221
Loss: 11.922098

**Making Predictions**


In [0]:
y = regressor.predict(input_fn=lambda: input_fn(prediction_set))
# .predict() returns an iterator; convert to a list and print predictions
predictions = list(itertools.islice(y, 6))
print ("Predictions: {}".format(str(predictions)))

In [0]:
Predictions: [ 33.30348587  17.04452896  22.56370163  34.74345398  14.55953979
  19.58005714]